In [7]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import os

# MENGAMBIL GESTUR 2 TANGAN

In [ ]:
# --- PROGRAM UTAMA ---
cap = cv2.VideoCapture(0)

# Detector lebih ketat supaya stabil
detector = HandDetector(maxHands=2, detectionCon=0.8, minTrackCon=0.7)

offset = 40  # supaya tidak kepotong jari

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)

    if len(hands) == 2:
        # Ambil bbox kedua tangan
        x1, y1, w1, h1 = hands[0]['bbox']
        x2, y2, w2, h2 = hands[1]['bbox']

        # Hitung container mengelilingi 2 tangan
        x_min = min(x1, x2) - offset
        y_min = min(y1, y2) - offset
        x_max = max(x1 + w1, x2 + w2) + offset
        y_max = max(y1 + h1, y2 + h2) + offset

        # Pastikan tidak keluar frame
        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(img.shape[1], x_max)
        y_max = min(img.shape[0], y_max)

        # Crop gabungan
        imgCrop = img[y_min:y_max, x_min:x_max]

        if imgCrop.size > 0:
            cv2.imshow("Gestur BISINDO", imgCrop)

        # Gambar kotak container di tampilan utama
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Tampilan utama dengan deteksi tangan
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# MENGAMBIL GESTUR 1 TANGAN

In [12]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1, detectionCon=0.8, minTrackCon=0.7)

offset =30
imgSize = 300

folder = 'datasets/I'

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)

    if hands:
        x, y, w, h = hands[0]['bbox']

        x_min = max(0, x - offset)
        y_min = max(0, y - offset)
        x_max = min(img.shape[1], x + w + offset)
        y_max = min(img.shape[0], y + h + offset)

        imgCrop = img[y_min:y_max, x_min:x_max]

        if imgCrop.size > 0:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal+wGap] = imgResize
            else:
                k = imgSize / h
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal+hGap, :] = imgResize

            cv2.imshow("Gestur BISINDO (1 Tangan)", imgCrop)
            cv2.imshow("White Image", imgWhite)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    elif key & 0xFF == ord('s'):
        if imgCrop.size > 0:
            count = len(os.listdir(folder))
            cv2.imwrite(f'{folder}/Image_{count+1}.jpg', imgWhite)
            print(f'Gambar disimpan: {folder}/Image_{count+1}.jpg')

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1757038992.206867   19932 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757038992.208810   25864 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-29-generic)
W0000 00:00:1757038992.235238   25847 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757038992.260520   25849 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Gambar disimpan: datasets/I/Image_1.jpg
Gambar disimpan: datasets/I/Image_2.jpg
Gambar disimpan: datasets/I/Image_3.jpg
Gambar disimpan: datasets/I/Image_4.jpg
Gambar disimpan: datasets/I/Image_5.jpg
Gambar disimpan: datasets/I/Image_6.jpg
Gambar disimpan: datasets/I/Image_7.jpg
Gambar disimpan: datasets/I/Image_8.jpg
Gambar disimpan: datasets/I/Image_9.jpg
Gambar disimpan: datasets/I/Image_10.jpg
Gambar disimpan: datasets/I/Image_11.jpg
Gambar disimpan: datasets/I/Image_12.jpg
Gambar disimpan: datasets/I/Image_13.jpg
Gambar disimpan: datasets/I/Image_14.jpg
Gambar disimpan: datasets/I/Image_15.jpg
Gambar disimpan: datasets/I/Image_16.jpg
Gambar disimpan: datasets/I/Image_17.jpg
Gambar disimpan: datasets/I/Image_18.jpg
Gambar disimpan: datasets/I/Image_19.jpg
Gambar disimpan: datasets/I/Image_20.jpg
Gambar disimpan: datasets/I/Image_21.jpg
Gambar disimpan: datasets/I/Image_22.jpg
Gambar disimpan: datasets/I/Image_23.jpg
Gambar disimpan: datasets/I/Image_24.jpg
Gambar disimpan: datasets